# Akar-Bazzi Algorithmic Complexity Calculator

# Table of Contents

* [1. Akra-Bazzi Theorem](#1.-Akra-Bazzi-Theorem)
    * [1.1 History](#1.1-History)
    * [1.2 Theorem Statement](#1.2-Theorem-Statement)
    
* [2. Recursion Depth](#2.-Recursion-Depth)
    * [2.1 Solving $p$](#2.1-Solving-$p$)
        * [2.1.1 Analytic Approach](#2.1.1-Analytic-Approach)
        * [2.1.2 Numerical Approach](#2.1.2-Numerical-Approach)
        * [2.1.3 Flaws with `sympy`](#2.1.3-Flaws-with-sympy)
    
* [3. Integral-Expression](#3.-Integral-Expression)
    * [3.1 Algorithmic-Complexity](#3.1-Algorithmic-Complexity)
        * [3.1.1 Rudimentary Complexity Calculator](#3.1.1-Rudimentary-Complexity-Calculator)
        * [3.1.2 The-Effects-of-$p$-and-$d$](#3.1.2-The-Effects-of-$p$-and-$d$)
        * [3.1.3 Refining the Akar-Bazzi Calculator for $p$ and $d$](#3.1.3-Refining-the-Akar-Bazzi-Calculator-for-$p$-and-$d$)
    * [3.2 Refactoring](#3.2-Refactoring)

* [4. Future Directions](#4.-Future-Directions)
    * [4.1 Additional Features](#4.1-Additional-Features)
    * [4.2 Possible Generalization](#4.2-Possible-Generalization)
    
* [5. Bibliography](#5.-Bibliography)

## 1. Akra-Bazzi Theorem

### 1.1 History

The application of the [Master Theorem](https://en.wikipedia.org/wiki/Master_theorem_(analysis_of_algorithms)) is limited to Divide & Conquer type recurrence relations for which each subproblem of a recursive invocation of an algorithm is of the same size.

The [Akra-Bazzi Theorem](https://en.wikipedia.org/wiki/Akra%E2%80%93Bazzi_method#:~:text=In%20computer%20science%2C%20the%20Akra,problems%20have%20substantially%20different%20sizes.) is a generalization of the Master Theorem for algorithms that can be modelled by Divide & Conquer (DAQ) type recurrence relations having different-size subproblems.

### 1.2 Theorem Statement

Given a recurrence relation of the form ...

$$
    T(n) = \sum_{i=1}^{k} {a_i T \left( \frac{n}{b_i} \right)} + O(n^d)
$$

with $a_i > 1$, $b_i > 1$ $\forall i$ and constant fixed values $k$ and $d$

Then, the solution of the recurrence is given by the integral expression ...

$$
    T(n) = n^p + n^p \int_{1}^{n} {\frac{u^d}{u^{p+1}}} du
$$

where $p$ is obtained by solving the equation ...

$$
    \sum_{i=1}^{k} a_i b_i^p = 1
$$

In [7]:
from sympy import *

In [8]:
from scipy.optimize import minimize

## 2. Recursion Depth

### 2.1 Solving $p$

The characterization for equations of the form

$$
    \sum_{i=1}^{k} a_i b_i^p = 1
$$

is not obvious - It can be classified as a weighted sum, a power sum or a general algebraic equation.

The Theorem statement requires us to solve the equation $\sum_{i=1}^{k} a_i b_i^p = 1$ for $p$ which, in some case, may be a dauntingly difficult task. We will first consider an analytic approach for the simple cases, for which `sympy` is a sufficient tool. Then we will consider a numerical approach, for the more complicated cases, for which `scipy` is a sufficient tool.

#### 2.1.1 Analytic Approach

In this section we build a python function that solves for the parameter $p$ using `sympy`

In [ ]:
def Rec():
    # Base Case
    if(n<=1):
        #stop
    
    2 * Rec(n/2) + 3 * Rec(n/3) + 1/2 * Rec(2/4)

Test Arrays ...

$$T(n) = T \left( \frac{n}{2} \right) + O(n^d)$$

In [9]:
# Corresponds to the recurrence relation above
a_i = [2]
b_i = [2]

In [10]:
a_i

[2]

In [11]:
b_i

[2]

Notice that the `length` of the arrays `a_i` and `b_i` are equinumerous, since there is a *one-to-one* correspondence between them.

This is a direct consequence of the form of the recurrence relation in the Theorem statement

In [12]:
length = len(a_i)

length

1

Instantiate symbol for parameter $p$ in symbolic expression

In [13]:
p = Symbol('p', finite=True)

In [14]:
p

p

$$\sum_{i=1}^{k} a_i b_i^p = \frac{2}{2^p}$$

In [15]:
expression = 0

for x in range(length):
    expression += a_i[x] / b_i[x]**p

In [16]:
expression

2/2**p

To solve the equation, we can use sympy's `solve()` method ...

In [17]:
solution = solve(Eq(expression, 1), p)

In [18]:
solution

[1]

Extract the solution ...

In [19]:
solution[0]

1

Putting it all together ...

In [20]:
def calculate_p(array_a_i, array_b_i):
    p = Symbol('p', finite=True)
    length = len(array_a_i)
    
    expression = 0

    for x in range(length):
        expression += a_i[x] / b_i[x]**p
    
    solution = solve(Eq(expression, 1), p)
    
    return solution[0]

Test solution ...

In [21]:
a_i = [2]
b_i = [2]

In [22]:
calculate_p(a_i, b_i)

1

#### 2.1.2 Numerical Approach

Sympy cannot solve all type of equations of the form

$$
    \sum_{i=1}^{k} a_i b_i^p = 1
$$

##### Example

No algorithm exists for solving the equation analytically/algebraically ...

$$
    \frac{2}{2^p} + \frac{1}{3^p} = 1
$$

because the equation *does not have an analytic solution* (as is **MOST** cases)

Equations of the form 

$$
    \sum_{i=1}^{k} a_i b_i^p = 1
$$

are [monotonic](https://en.wikipedia.org/wiki/Monotonic_function) and thus a unique solution exists, but it does not give us insight to how it can be solved.

So the most general way in which we can approach solving equations of this type, is to employ numerical techniques to approximate the root.

Using `sympy` for equations of this type has dire consequences ...

In [23]:
a_i = [2, 1]
b_i = [2, 3]

In [24]:
calculate_p(a_i, b_i)

NotImplementedError: multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p

So we will do exception handling to treat the case for which the `calculate_p()` method does not work!

In [ ]:
a_i = [2, 1]
b_i = [2, 3]

In [25]:
a_i

[2, 1]

In [26]:
b_i

[2, 3]

In [27]:
length = len(a_i)

length

2

In [28]:
p = Symbol('p', finite=True)

In [29]:
expression = 0

for x in range(length):
    expression += a_i[x] / b_i[x]**p

In [30]:
expression

3**(-p) + 2/2**p

First we need to convert the symbolic expression to a python function so that we can use numerical metods to find the optimal solution.

To do this, we use `sympy.lambdify(<variable>, <expression>)`

In [31]:
f = lambdify(p, expression) # first parameter is the variable, second parameter is the expression to convert

In [32]:
f

<function _lambdifygenerated(p)>

Notice that we can still recover the symbolic representation ...

In [33]:
f(p)

3**(-p) + 2/2**p

Test cases:

Notice that we want $f(p)$ to be as close to $1$ as possible since the equation to be solved is $\frac{2}{2^p} + \frac{1}{3^p} = 1$ with $f(p)$ on the Lef-Hand side!

In [34]:
f(1)

1.3333333333333333

In [35]:
f(2)

0.6111111111111112

So we might "guess" the optimal value between 1 and 2

In [36]:
f(1.1)

1.2316858114837281

In [37]:
f(1.2)

1.1381310838828678

In [38]:
f(1.3)

1.0519934274645235

In [39]:
f(1.4)

0.9726562882476172

In [40]:
f(1.35)

1.0115111681129008

In [41]:
f(1.36)

1.0036122456542467

In [42]:
f(1.37)

0.9957778485027282

In [43]:
f(1.365)

0.9996870161939175

In [44]:
f(1.364)

1.0004707749256907

The above *refinement* process was not to difficult, so let's see if we can incorporate this in the `calculate_p()` method by using `scipy`

In [45]:
# from scipy.optimize import minimize
# already imported at top of script

Use a numerical optimization library to calculate $p$

In [46]:
# Define the objective function to minimize
def objective_function(p):
    return abs(f(p) - 1.0)

In [47]:
objective_function(p)

Abs(-1.0 + 3**(-p) + 2/2**p)

Refinement for the guess of the root was $p = 1.364$

In [48]:
# Use scipy's minimize function to find the optimal p
result = minimize(objective_function, [1.364]) # guess: 1.364

In [49]:
# Extract the optimal p
p = result.x[0]

In [50]:
p

1.3646005572584177

Notice how good our guess was, based on observation alone ...

Some basic [error analysis](https://en.wikipedia.org/wiki/Error_analysis_(mathematics)) checks for the numerical approximation ...

In [51]:
p_round = round(p, 2)

In [52]:
p_round

1.36

In [53]:
abs(p - p_round)

0.00460055725841757

In [54]:
f(p)

1.0000000058389757

In [55]:
f(p_round)

1.0036122456542467

In [56]:
abs(f(p) - f(p_round))

0.0036122398152709234

In [57]:
approx_root = round(f(p_round), 2)

In [58]:
approx_root

1.0

In [59]:
abs(f(p)-approx_root)

5.838975747352038e-09

In [60]:
round(abs(f(p)-approx_root), 2)

0.0

#### Refining `calculate_p()`

$$\sum_{i=1}^{k} a_i b_i^p = 1$$

Setting up the exception handling (needs work)

In [61]:
def calculate_p(array_a_i, array_b_i):
    p = Symbol('p', finite=True)
    length = len(array_a_i)
    
    expression = 0

    for x in range(length):
        expression += a_i[x] / b_i[x]**p
    
    try:
        solution = solve(Eq(expression, 1), p)
   
    except NotImplementedError as e:
        print(e)
        
        # scipy stuff ...
        solution = [0] # fake temporary solution for testing exception message
    
    return solution[0]

In [62]:
a_i = [2]
b_i = [2]

$$\frac{2}{2^p} = 1$$

In [63]:
calculate_p(a_i, b_i)

1

The above result confirms that the `calculate_p()` method still works for simpler equations solvable by using `sympy`

In [64]:
a_i = [2, 1]
b_i = [2, 3]

$$\frac{2}{2^p} + \frac{1}{3^p} = 1$$

In [65]:
calculate_p(a_i, b_i)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p


0

Notice that the above result is not the correct approximate solution and was hard-coded for testing purposes only ...

The above result confirms that the `exception handling` is treated correctly followed by an alternative approach ...

Defining the code inside the `excpetion block` ...

In [66]:
a_i = [2, 1]
b_i = [2, 3]

$$\frac{2}{2^p} + \frac{1}{3^p} = 1$$

In [67]:
p = Symbol('p', finite=True)

expression = 0
for x in range(length):
    expression += a_i[x] / b_i[x]**p

f = lambdify(p, expression)

def objective_function(p):
    return abs(f(p) - 1.0)

guess = 1

result = minimize(objective_function, [guess])
solution = result.x[0]

solution

1.3646005630226317

The above result is the correct approximate solution as seen before ...

In [68]:
def calculate_p(array_a_i, array_b_i):
    p = Symbol('p', finite=True)
    length = len(array_a_i)
    
    expression = 0

    for x in range(length):
        expression += a_i[x] / b_i[x]**p
    
    try:
        solution = solve(Eq(expression, 1), p)
   
    except NotImplementedError as e:
        print(e)
        f = lambdify(p, expression)

        def objective_function(p):
            return abs(f(p) - 1.0)

        guess = 1

        result = minimize(objective_function, [guess])        
        
        solution = result.x
        for sol in result.x:
            round(sol)        
    
    return solution[0]

In [69]:
a_i = [2, 1]
b_i = [2, 3]

In [70]:
calculate_p(a_i, b_i)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p


1.3646005630226317

Add the functionality to prompt the user for an initial guess for the root of the equation - Not a strict requirement!

In [73]:
print("Please enter an initial guess (float) for the root of the equation ... ")

guess = float(input())

print(guess)

Please enter an initial guess (float) for the root of the equation ... 
2
2.0


In [74]:
def calculate_p(array_a_i, array_b_i):
    p = Symbol('p', finite=True)
    length = len(array_a_i)
    
    expression = 0

    for x in range(length):
        expression += a_i[x] / b_i[x]**p
    
    try:
        solution = solve(Eq(expression, 1), p)
   
    except NotImplementedError as e:
        print(e)
        f = lambdify(p, expression)

        def objective_function(p):
            return abs(f(p) - 1.0)
        
        print("Guess: ")
        guess = float(input())
        
        result = minimize(objective_function, [guess])        
        
        solution = result.x
        for sol in result.x:
            round(sol)
    
    return solution[0]

#### Example 1

In [75]:
a_i = [2, 1]
b_i = [2, 3]

$$\frac{2}{2^p} + \frac{1}{3^p} = 1$$

In [76]:
calculate_p(a_i, b_i)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p
Guess: 
1


1.3646005630226317

#### Example 2

$$\frac{2}{2^p}= 1$$

In [77]:
a_i = [2]
b_i = [2]

In [78]:
calculate_p(a_i, b_i)

1

#### Example 3

In [79]:
a_i = [1]
b_i = [2]

$$\frac{1}{2^p}= 1$$

In [80]:
calculate_p(a_i, b_i)

0

#### Example 4

In [81]:
a_i = [3]
b_i = [2]

$$\frac{3}{2^p} = 1$$

In [82]:
calculate_p(a_i, b_i)

log(3)/log(2)

Notice the above solution is equivalent to $\log_2{3}$

#### Example 5

In [83]:
a_i = [2, 1, 2]
b_i = [2, 3, 3]

$$\frac{2}{2^p} + \frac{1}{3^p} + \frac{2}{3^p} = 1$$

notice that the value of the initial guess for $p$ can have a great influence on the refined result ...

For example, the initial guess $p=1$ refines to $p = 1.7878849085670687 \dots$

In [86]:
option1 = calculate_p(a_i, b_i)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3/3**p + 2/2**p
Guess: 
1


In [87]:
option1

1.7878849085670687

Whereas the initial guess $p = 12$ refines to $p = 11.647079467773438 \dots$

In [88]:
option12 = calculate_p(a_i, b_i)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3/3**p + 2/2**p
Guess: 
12


In [1414]:
option12

11.647079467773438

... so it is always important to try and make an educated guess by first to establish a reasonable interval in which the solution may be found!

The result for $p$ should always be tested when using a numerical approach as the below computations illustrate!

In [89]:
expression = 0

for x in range(length):
    expression += a_i[x] / b_i[x]**p

In [90]:
expression

3**(-p) + 2/2**p

In [91]:
f = lambdify(p, expression)

In [92]:
f

<function _lambdifygenerated(p)>

In [93]:
f(p)

3**(-p) + 2/2**p

In [94]:
f(option1)

0.7194617383876748

In [95]:
f(option12)

0.0006263780583892548

#### Example 6

In [1422]:
a_i = [1, 2, 3]
b_i = [2, 3, 4]

$$\frac{1}{2^p} + \frac{2}{3^p} + \frac{3}{4^p} = 1$$

The initial guess $p=1$ refines to $p = 2.01 \dots$

In [1423]:
calculate_p(a_i, b_i)

multiple generators [12**p, 24**p, 6**p, 8**p]
No algorithms are implemented to solve equation -1 + 3/4**p + 2/3**p + 2**(-p)
Guess: 
1


2.01

 ... while the initial guess $p=2$ refines to $p = 1.6011213861678517 \dots$

In [1426]:
calculate_p(a_i, b_i)

multiple generators [12**p, 24**p, 6**p, 8**p]
No algorithms are implemented to solve equation -1 + 3/4**p + 2/3**p + 2**(-p)
Guess: 
2


1.6011213861678517

#### Flaws with Sympy

In [1428]:
a_i = [1, 1]
b_i = [2, 3]

$$\frac{1}{2^p} + \frac{1}{3^p} = 1$$

In [1429]:
calculate_p(a_i, b_i)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2**(-p)
Guess: 
1


0.7878849080302573

#### 2.1.3 Flaws with `sympy`

Besides the "traps" that we have to look out for when using the `calculate_p()` method for approximating the solution numerically, we still face quite a bit of trouble with optimizing the result given by `sympy` as the following example illustrates ...

#### Example

In [1430]:
a_i = [1, 1]
b_i = [2, 4]

$$\frac{1}{2^p} + \frac{1}{4^p} = 1$$

As the `calculate_p()` method is currently defined, it's still not perfect, even for those equations that *can* be solved analytically by `sympy`

In [1431]:
calculate_p(a_i, b_i)

(log(-1/2 + sqrt(5)/2) + I*pi)/log(2)

The result is a complex number ...

In [1432]:
N(calculate_p(a_i, b_i))

-0.694241913630617 + 4.53236014182719*I

In [1433]:
re(N(calculate_p(a_i, b_i)))

-0.694241913630617

The equation $\frac{1}{2^p} + \frac{1}{4^p} = 1$ actually has two solutions, one positive and the other negative as can be obtained by manipulating it as follows ...

$$
\begin{align}
    &\frac{1}{2^p} + \frac{1}{4^p} = 1 \\
    &\left( \frac{1}{2} \right)^{p} + \left( \frac{1}{2} \right)^{2p} = 1 \\
\end{align}
$$

<p style = 'text-align: center'> ... by substituting $x$ for $\left( \frac{1}{2} \right)^{p}$ we get ... </p>

$$
\begin{align}
    x^2 + x &= 1 \\
    x^2 + x -1 &= 0 \\
\end{align}
$$

<p style = 'text-align: center'> ... then using the <a href='https://en.wikipedia.org/wiki/Quadratic_formula'>quadratic formula</a> ... </p>

$$
\begin{align}
    x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} \\
\end{align}
$$

<p style = 'text-align: center'> ... we get ... </p>

$$
    x = \frac{-1 + \sqrt{5}}{2} \quad \text{or} \quad x = \frac{-1 - \sqrt{5}}{2}
$$

<p style = 'text-align: center'> ... but $x$ is $\left( \frac{1}{2} \right)^{p}$ so substituting back and solving for $p$ ... </p>

<p style = 'text-align: center'> ... we get that after rationalizing the denominators ... </p>

$$
    p = \log_2\left(\frac{1}{2}\left(1+\sqrt{5}\right)\right) \quad \text{or} \quad p = \log_2\left(\frac{1}{2}\left(1-\sqrt{5}\right)\right)
$$

But the solution for $p$ given by ...

$$
    p = \log_2\left(\frac{1}{2}\left(1-\sqrt{5}\right)\right)
$$

... does not make sense in this context, because the result would be a [Complex Logarithm](https://en.wikipedia.org/wiki/Complex_logarithm#Definition_of_principal_value), hence the `sympy` result as obtained before!

So, that being said, we are only interested in the positive values for $p$, since $p$ represents "recursive depth" and the steps for recursive calls are forward in "time" not "backwards"

Let's develop the formalism to incorporate this into our code for the `calculate_p()` method!

To improve our `calculate_p()` method to handle this situation is not a hard task at all. Indeed, we can specify within the symbol decleration for $p$ that it must be positive, by including the attribute `nonnegative=True`

In [1434]:
def calculate_p(array_a_i, array_b_i):
    p = Symbol('p', finite=True, nonnegative=True, imaginary=False) # Set Attribute positive = true
    length = len(array_a_i)
    
    expression = 0

    for x in range(length):
        expression += a_i[x] / b_i[x]**p
    
    try:
        solution = solve(Eq(expression, 1), p)
   
    except NotImplementedError as e:
        print(e)
        f = lambdify(p, expression)

        def objective_function(p):
            return abs(f(p) - 1.0)
        
        print("Guess: ")
        guess = float(input())
        
        result = minimize(objective_function, [guess])
        
        solution = result.x
        for sol in result.x:
            round(sol)
    
    return solution[0]

In [1435]:
a_i = [1, 1]
b_i = [2, 4]

In [1436]:
calculate_p(a_i, b_i)

-1 + log(1 + sqrt(5))/log(2)

Thus we get out beautiful result, just as expected!

In [1437]:
N(calculate_p(a_i, b_i))

0.694241913630617

The predicates/attributes associated with the symbolic parameter $p$ ***must*** be as follows ...

In [1438]:
p = Symbol('p', nonnegative=True, imaginary=False, finite=True)

The full set of know predicates for a symbol can be accessed using `.assumptions0` ...

In [1439]:
p.assumptions0

{'nonnegative': True,
 'real': True,
 'negative': False,
 'infinite': False,
 'extended_real': True,
 'extended_nonnegative': True,
 'extended_negative': False,
 'complex': True,
 'imaginary': False,
 'hermitian': True,
 'commutative': True,
 'finite': True}

See [here](https://docs.sympy.org/latest/guides/assumptions.html) for more information!

## 3. Integral-Expression

In the previous section we developed a method `calculate_p()` for determining the parameter $p$ in the Integral expression given by the Akar-Bazzi Theorem ...

$$
    T(n) = n^p + n^p \int_{1}^{n} {\frac{u^d}{u^{p+1}}} du
$$

... such that $p$ satisfies the equation ...

$$\sum_{i=1}^{k} a_i b_i^p = 1$$

So most of the heavy work has already been done!

The remainder of our task is to define a procedure using `sympy` which will perform the intergration for us to compute the algorithmic complexity of an any Divide & Conquer recurrence relation of the form ...

$$
    T(n) = \sum_{i=1}^{k} {a_i T \left( \frac{n}{b_i} \right)} + O(n^d)
$$

... given, as input, an array of the parameters $a_i$ and $b_i$

### 3.1 Algorithmic-Complexity

In this section we define a procedure that solves the integral in symbolic form (if possible) with parameters $u$ and $n$ and finally combine the terms for a result!

Recall that $d$ represents the exponent of the polynomial function outside of the recursive calls $O(n^d)$ ...

In [1440]:
u = Symbol('u', finite=True)
n = Symbol('n', finite=True)

In [1441]:
p = Symbol('p', nonzero=False, negative=False, imaginary=False)
d = Symbol('d', positiveorzero=True, real=True, finite=True)

Thus we can define integrand ...

$${\frac{u^d}{u^{p+1}}}$$

... in sympy, as follows ...

In [1442]:
integrand = u**d / u**(p+1)

In [1443]:
integrand

u**d*u**(-p - 1)

Hence, the integral ...

$$\int_{1}^{n} {\frac{u^d}{u^{p+1}}} du$$

... can be define symbolically as ...

In [1444]:
integral = integrate(integrand, (u, 1, n))

In [1445]:
integral

Piecewise((-n**d/(-d*n**p + n**p*p) + 1/(-d + p), Ne(d, p)), (p*log(n)/(p + 1) + log(n)/(p + 1), True))

#### 3.1.1 Rudimentary Complexity Calculator

Below we have a rudimentary implementation of the procedure in development for calculating the complexity of an algorithm using the Akar-Bazzi Theorem

In [1446]:
def calculate_complexity(array_a_i, array_b_i, d):
    
    n = Symbol('n', positive=True, finite=True, integer=True)
    u = Symbol('u')
    
    power = calculate_p(array_a_i, array_b_i) # From the previous section

    integral = integrate(u**d / u**(power+1), (u, 1, n))

    result = expand(n**power + n**power * integral)
    
    return result

#### Example 1

The recurrence relation ...

$$
    T(n) = 2T(n/2) + O(1)
$$

... has parameters corresponding to the values below ...

In [1447]:
d = 0
a_i = [2]
b_i = [2]

In [1448]:
calculate_p(a_i, b_i)

1

... and the corresponding complexity is obtained by computing the integral ...

$$
    T(n) = n^1 + n^1 \int_{1}^{n} {\frac{u^0}{u^{1+1}}} du
$$

In [1449]:
calculate_complexity(a_i, b_i, d)

2*n - 1

Which is clearly $O(n)$ Linear Runtime

Notice that ...

In [1453]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.One

#### Example 2

The recurrence relation ...

$$
    T(n) = 2T(n/2) + O(n)
$$

... has parameters correspondind to the values below ...

In [1454]:
d = 1
a_i = [2]
b_i = [2]

In [1455]:
calculate_p(a_i, b_i)

1

... and the corresponding complexity is obtained by computing the integral ...

$$
    T(n) = n^1 + n^1 \int_{1}^{n} {\frac{u^1}{u^{1+1}}} du
$$

In [1456]:
calculate_complexity(a_i, b_i, d)

n*log(n) + n

Which is clearly $O(n*\log(n))$ Linearithmic Runtime

Notice that ... 

In [1457]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.One

... we may need this fact later ...

#### Example 3

The recurrence relation ...

$$
    T(n) = T(n/2) + O(1)
$$

... has parameters corresponding to the values below ...

In [1458]:
d = 0
a_i = [1]
b_i = [2]

In [1459]:
calculate_p(a_i, b_i)

0

... and the corresponding complexity is obtained by computing the integral ...

$$
    T(n) = n^0 + n^0 \int_{1}^{n} {\frac{u^0}{u^{0+1}}} du
$$

In [1460]:
calculate_complexity(a_i, b_i, d)

log(n) + 1

Which is clearly $O(\log(n))$ Logarithmic Runtime

Notice that ...

In [1462]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Zero

#### Example 4

The recurrence relation ...

$$
    T(n) = 2T(n/2) + O(n^2)
$$

... has parameters corresponding to the values below ...

In [1463]:
d = 2
a_i = [2]
b_i = [2]

In [1464]:
calculate_p(a_i, b_i)

1

... and the corresponding complexity is obtained by computing the integral ...

$$
    T(n) = n^1 + n^1 \int_{1}^{n} {\frac{u^2}{u^{1+1}}} du
$$

In [1465]:
calculate_complexity(a_i, b_i, d)

n**2

Which is clearly $O(n^2)$ Quadratic Runtime

Notice that ...

In [1466]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.One

#### Example 5

The recurrence relation ...

$$
    T(n) = 2T(n/2) + O(\sqrt{n})
$$

... has parameters corresponding to the values below ...

In [1467]:
d = Rational(1/2)
a_i = [1]
b_i = [2]

In [1468]:
calculate_p(a_i, b_i)

0

... and the corresponding complexity is obtained by computing the integral ...

$$
    T(n) = n^1 + n^1 \int_{1}^{n} {\frac{u^{\frac{1}{2}}}{u^{1+1}}} du
$$

In [1469]:
calculate_complexity(a_i, b_i, d)

2*sqrt(n) - 1

Which is clearly $O(\sqrt{n})$ Square-Root Runtime

Notice that ...

In [1470]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Zero

#### Example 6

The recurrence relation ...

$$
    T(n)=4T\left(\frac{n}{2}\right)+n
$$

... has parameters corresponding to the values below ...

In [1471]:
d = 1
a_i = [4]
b_i = [2]

In [1472]:
calculate_p(a_i, b_i)

2

In [1473]:
calculate_complexity(a_i, b_i, d)

2*n**2 - n

Which is clearly $O(n^2)$ Quadratic Runtime

Notice that ...

In [1474]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Integer

Example 7

In [1475]:
d = 0
a_i = [2]
b_i = [4]

In [1476]:
calculate_p(a_i, b_i)

1/2

In [1477]:
calculate_complexity(a_i, b_i, d)

3*sqrt(n) - 2

Which is clearly $O(\sqrt{n})$ Square-Root Runtime

In [1479]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Half

Unfortunately, not all recurrence relations are so simple as the preceding 4 examples.

The not-so-simple recurrence relations have very strange results, as we shall explore next ...

#### 3.1.2 The Effects of $p$ and $d$

There are, as we shall see, two contributing factors, that determine whether an integral expression appears *"inelegant"*

#### Horrible Example 1

Certain values of $p$ although calculated analytically, may render the integral difficult to read, depending on its type ...

In [1494]:
d = 1
a_i = [1, 1]
b_i = [2, 4]

In [1495]:
calculate_p(a_i, b_i)

-1 + log(1 + sqrt(5))/log(2)

In [1496]:
# Not much of an effect ...
simplify(calculate_p(a_i, b_i))

-1 + log(1 + sqrt(5))/log(2)

In [1497]:
calculate_complexity(a_i, b_i, d)

n**(log(1 + sqrt(5))/log(2))*Integral(Piecewise((-u*log(1 + sqrt(5))*gamma(-log(1 + sqrt(5))/log(2) + 2)/(u**(log(1 + sqrt(5))/log(2))*log(2)*gamma(-log(1 + sqrt(5))/log(2) + 3)) + 2*u*gamma(-log(1 + sqrt(5))/log(2) + 2)/(u**(log(1 + sqrt(5))/log(2))*gamma(-log(1 + sqrt(5))/log(2) + 3)), (u < 1) | (1/u < 1)), (meijerg(((0,), (-log(1 + sqrt(5))/log(2) + 3,)), ((-log(1 + sqrt(5))/log(2) + 2,), (0,)), u)/u + meijerg(((-log(1 + sqrt(5))/log(2) + 2, 1), ()), ((), (-log(1 + sqrt(5))/log(2) + 2, 0)), u)/u - meijerg(((-log(1 + sqrt(5))/log(2) + 3, 1), ()), ((), (-log(1 + sqrt(5))/log(2) + 2, 0)), u)*log(1 + sqrt(5))/(u*log(2)) + 2*meijerg(((-log(1 + sqrt(5))/log(2) + 3, 1), ()), ((), (-log(1 + sqrt(5))/log(2) + 2, 0)), u)/u, True)), (u, 1, n))/n + n**(log(1 + sqrt(5))/log(2))/n

There is still an easy work-around for this problem, because we can obtain a numerical value for $p$ much in the same way as we have done before ...

In [1498]:
integral = integrate(u**d / u**(power+1), (u, 1, n))

result = expand(n**power + n**power * integral)

result

-2.27055945395966*n**0.694241913630617 + 3.27055945395966*n**1.0

We take it that anyone would agree - the above expression is much easier on the eye for deciding that the recurrence relation models a Linear $O(n)$ algorithmic runtime!

Notice that ...

In [1488]:
type(calculate_p(a_i, b_i))

sympy.core.add.Add

#### Horrible Example 2

Certain values of $d$ may render the integral difficult to read, depending on its type ...

In [1518]:
d = log(3)/log(2)
a_i = [2]
b_i = [2]

Notice that ...

In [1519]:
type(d)

sympy.core.mul.Mul

In [1520]:
calculate_p(a_i, b_i)

1

Notice that ...

In [1521]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.One

In [1522]:
# Not much of an effect ...
simplify(d)

log(3)/log(2)

Notice that ...

In [1523]:
type(d)

sympy.core.mul.Mul

In [1524]:
calculate_complexity(a_i, b_i, d)

n*Integral(Piecewise((-u**(log(3)/log(2))*gamma(-1 + log(3)/log(2))/(u**2*gamma(log(3)/log(2))) + u**(log(3)/log(2))*log(3)*gamma(-1 + log(3)/log(2))/(u**2*log(2)*gamma(log(3)/log(2))), (u < 1) | (1/u < 1)), (meijerg(((0,), (log(3)/log(2),)), ((-1 + log(3)/log(2),), (0,)), u)/u - meijerg(((log(3)/log(2), 1), ()), ((), (-1 + log(3)/log(2), 0)), u)/u + meijerg(((log(3)/log(2), 1), ()), ((), (-1 + log(3)/log(2), 0)), u)*log(3)/(u*log(2)) + meijerg(((-1 + log(3)/log(2), 1), ()), ((), (-1 + log(3)/log(2), 0)), u)/u, True)), (u, 1, n)) + n

Again, a numerical approach will work ...

In [1525]:
d = N(d)
d

1.58496250072116

In [1526]:
integral = integrate(u**d / u**(power+1), (u, 1, n))

result = expand(n**power + n**power * integral)

result

-0.122686524251576*n**0.694241913630617 + 1.12268652425158*n**1.58496250072116

The above result is much easier on the eye as the previous integral!

Notice that ...

In [1527]:
type(d)

sympy.core.numbers.Float

The type `sympy.core.mul.Mul` and `sympy.core.add.Add` is clearly different from `sympy.core.numbers.Integer` or `sympy.core.numbers.Half` or `sympy.core.numbers.Zero` or `sympy.core.numbers.One`

we will use this distinction of types as a criteria for deciding how the integral should be computed, i.e. numerically or symbolically!

#### Horrible Example 3

Any combination of values of $p$ or $d$, may render the integral hard to read, depending on their type ...

In [1552]:
d = log(3)/log(2)
a_i = [1, 1]
b_i = [2, 4]

In [1553]:
d

log(3)/log(2)

Notice that ...

In [1554]:
type(d)

sympy.core.mul.Mul

In [1555]:
simplify(d)

log(3)/log(2)

In [1534]:
type(simplify(d))

sympy.core.mul.Mul

In [1535]:
calculate_p(a_i, b_i)

-1 + log(1 + sqrt(5))/log(2)

In [1536]:
type(calculate_p(a_i, b_i))

sympy.core.add.Add

In [1537]:
calculate_complexity(a_i, b_i, d)

n**(log(1 + sqrt(5))/log(2))*Integral(Piecewise((-u**(log(6)/log(2))*log(1 + sqrt(5))*gamma(-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2))/(u*u**(log(1 + sqrt(5))/log(2))*log(2)*gamma(-log(1 + sqrt(5))/log(2) + log(3)/log(2) + 2)) + u**(log(6)/log(2))*log(6)*gamma(-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2))/(u*u**(log(1 + sqrt(5))/log(2))*log(2)*gamma(-log(1 + sqrt(5))/log(2) + log(3)/log(2) + 2)), (u < 1) | (1/u < 1)), (meijerg(((0,), (-log(1 + sqrt(5))/log(2) + log(3)/log(2) + 2,)), ((-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2),), (0,)), u)/u + meijerg(((-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2), 1), ()), ((), (-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2), 0)), u)/u - meijerg(((-log(1 + sqrt(5))/log(2) + log(3)/log(2) + 2, 1), ()), ((), (-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2), 0)), u)*log(1 + sqrt(5))/(u*log(2)) + meijerg(((-log(1 + sqrt(5))/log(2) + log(3)/log(2) + 2, 1), ()), ((), (-log(1 + sqrt(5))/log(2) + 1 + log(3)/log(2), 0)), u)/u + meijerg(((-log(1 + sqrt(5))/log(

Both $d$ and $p$ are the reason that the above integral is difficult to read!

In [1538]:
d = N(d)
d

1.58496250072116

In [1540]:
type(d)

sympy.core.numbers.Float

In [1541]:
power = N(calculate_p(a_i, b_i))
power

0.694241913630617

In [1542]:
type(power)

sympy.core.numbers.Float

In [1543]:
integral = integrate(u**d / u**(power+1), (u, 1, n))

result = expand(n**power + n**power * integral)

result

-0.122686524251576*n**0.694241913630617 + 1.12268652425158*n**1.58496250072116

#### 3.1.3 Refining the Akar-Bazzi Calculator for $p$ and $d$

What we would like to achieve is to avoid obtaining complex and unreadable integral expressions as seen in the previous examples ...

In the previous subsection we observed that:

The integral result will be *elegant* if the type of $p$ and $d$ are not any of ...

1. `sympy.core.mul.Mul`

or

2. `sympy.core.add.Add`

So we do a test/check based on the *type* of the expression for $p$ using `==` and cast the result to a numerical value if necessary ...

In [1545]:
type(1)

int

In [1546]:
type(log(2))

log

In [1547]:
type(2*log(2))

sympy.core.mul.Mul

In [1548]:
type(log(2)/log(3))

sympy.core.mul.Mul

In [1551]:
type(3/2)

float

In [570]:
type(Rational(1/2))

sympy.core.numbers.RationalConstant

In [1558]:
type(-1 + log(1+sqrt(5))/log(2))

sympy.core.add.Add

In [1563]:
type(1+log(2))

sympy.core.add.Add

In [1564]:
def calculate_complexity(array_a_i, array_b_i, d):
    
    n = Symbol('n', positive=True, finite=True, integer=True)
    u = Symbol('u')
    
    power = calculate_p(array_a_i, array_b_i)
    
    # test for base type of p
    if (
            type(power) == type(log(2)/log(3))
            or
            type(power) == type(1+log(2))
       ):
        
        ##`sympy.core.numbers.RationalConstant`
        print("Symbolic Integral Unsatisfactory - convert p to numerical value")
        power = N(power) # cast p to numerical value

    integral = integrate(u**d / u**(power+1), (u, 1, n))

    result = expand(n**power + n**power * integral)
    
    return result

#### Example 1

In [1565]:
d = 1
a_i = [4]
b_i = [2]

In [1566]:
power = calculate_p(a_i, b_i)
power

2

In [1568]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Integer

In [1569]:
type(d)

int

In [1570]:
calculate_complexity(a_i, b_i, d)

2*n**2 - n

In [1571]:
type(1+log(2))

False

In [1572]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

False

#### Example 2

In [1573]:
d = 1
a_i = [2]
b_i = [2]

In [1574]:
power = calculate_p(a_i,b_i)
power

1

In [1575]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.One

In [1576]:
type(d)

int

In [1577]:
calculate_complexity(a_i, b_i, d)

n*log(n) + n

In [1578]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

False

#### Example 3

In [1579]:
d = 1
a_i = [1]
b_i = [2]

In [1580]:
power = calculate_p(a_i, b_i)
power

0

In [1582]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Zero

In [1584]:
type(d)

int

In [1585]:
calculate_complexity(a_i, b_i, d)

n

In [1586]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

False

#### Example 4

In [1587]:
d = 1
a_i = [2]
b_i = [4]

In [1588]:
power = calculate_p(a_i, b_i)
power

1/2

In [1589]:
type(calculate_p(a_i, b_i))

sympy.core.numbers.Half

In [1590]:
type(d)

int

In [1591]:
calculate_complexity(a_i, b_i, d)

-sqrt(n) + 2*n

In [1592]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

False

#### Example 5

In [1593]:
d = 1
a_i = [1, 1]
b_i = [2, 4]

In [1594]:
power = calculate_p(a_i, b_i)
power

-1 + log(1 + sqrt(5))/log(2)

In [1596]:
type(calculate_p(a_i, b_i))

sympy.core.add.Add

In [1597]:
type(d)

int

In [1598]:
calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-2.27055945395966*n**0.694241913630617 + 3.27055945395966*n**1.0

In [1599]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

True

#### Example 6

In [1600]:
d = 1
a_i = [2, 1]
b_i = [2, 3]

In [1601]:
power = calculate_p(a_i, b_i)
power

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p
Guess: 
1


1.3646005630226317

In [1602]:
type(calculate_p(a_i, b_i))

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p
Guess: 
1


numpy.float64

In [1603]:
type(d)

int

In [1604]:
calculate_complexity(a_i, b_i, d)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 3**(-p) + 2/2**p
Guess: 
1


-2.74272752545894*n**1.0 + 3.74272752545894*n**1.36460056302263

In [1605]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

False

Notice that for the given parameters of this problem, it does not matter whether we obtain `true` or `false` for the above boolean expression, since the `calculate_p()` method takes precedence over the "check" and therefore, if a numerical value for $p$ was returned by the `calculate_p()` method, then the integral will automatically be computed in a "numerical" form!

#### Example 7

In [1614]:
d = 1
a_i = [1, 2, 3]
b_i = [2, 3, 4]

In [1615]:
power = calculate_p(a_i, b_i)
power

multiple generators [12**p, 24**p, 6**p, 8**p]
No algorithms are implemented to solve equation -1 + 3/4**p + 2/3**p + 2**(-p)
Guess: 
2


1.6011213861678517

In [1616]:
type(calculate_p(a_i, b_i))

multiple generators [12**p, 24**p, 6**p, 8**p]
No algorithms are implemented to solve equation -1 + 3/4**p + 2/3**p + 2**(-p)
Guess: 
2


numpy.float64

In [1617]:
type(d)

int

In [1618]:
calculate_complexity(a_i, b_i, d)

multiple generators [12**p, 24**p, 6**p, 8**p]
No algorithms are implemented to solve equation -1 + 3/4**p + 2/3**p + 2**(-p)
Guess: 
2


-1.66355751602018*n**1.0 + 2.66355751602018*n**1.60112138616785

In [1619]:
type(power) == type(log(2)/log(3)) or type(power) == type(1+log(2))

False

#### Example 8 (Horrible effect of $d$)

In [1620]:
d = log(3)/log(2)
a_i = [2]
b_i = [2]

In [1621]:
d

log(3)/log(2)

In [1622]:
power = calculate_p(a_i, b_i)
power

1

In [1623]:
type(power)

sympy.core.numbers.One

In [1624]:
type(d)

sympy.core.mul.Mul

In [1625]:
power = calculate_p(a_i, b_i)
power

1

In [1626]:
calculate_complexity(a_i, b_i, d)

n*Integral(Piecewise((-u**(log(3)/log(2))*gamma(-1 + log(3)/log(2))/(u**2*gamma(log(3)/log(2))) + u**(log(3)/log(2))*log(3)*gamma(-1 + log(3)/log(2))/(u**2*log(2)*gamma(log(3)/log(2))), (u < 1) | (1/u < 1)), (meijerg(((0,), (log(3)/log(2),)), ((-1 + log(3)/log(2),), (0,)), u)/u - meijerg(((log(3)/log(2), 1), ()), ((), (-1 + log(3)/log(2), 0)), u)/u + meijerg(((log(3)/log(2), 1), ()), ((), (-1 + log(3)/log(2), 0)), u)*log(3)/(u*log(2)) + meijerg(((-1 + log(3)/log(2), 1), ()), ((), (-1 + log(3)/log(2), 0)), u)/u, True)), (u, 1, n)) + n

Up to now, we have only taken care of half the problem, i.e. the effect of $p$ on the final integral expression.

From the last example we observe that we still need refine the code of the `calculate_complexity()` method to take into account the effect of $d$ on the final integral expression as well!

As with $p$, here we also use `==` to test for the types on the expression for $d$

In [1627]:
def calculate_complexity(array_a_i, array_b_i, d):
    
    n = Symbol('n', positive=True, finite=True, integer=True)
    u = Symbol('u')
    
    power = calculate_p(array_a_i, array_b_i)
    
    # test for base type of p or d
    if (
            (
                type(power) == type(log(2)/log(3))
                or
                type(power) == type(1+log(2))
            )
        
            or
        
            (
                type(d) == type(log(2)/log(3))
                or
                type(d) == type(1+log(2))
            )
       ):
        
        ##`sympy.core.numbers.RationalConstant`
        print("Symbolic Integral Unsatisfactory - convert p to numerical value")
        power = N(power) # cast p to numerical value
        
        # convert d to numeric type
        d = N(d)

    integral = integrate(u**d / u**(power+1), (u, 1, n))

    result = expand(n**power + n**power * integral)
    
    return result

#### Example 1

In [1628]:
d = 1
a_i = [2]
b_i = [2]

In [1629]:
calculate_complexity(a_i, b_i, d)

n*log(n) + n

In [1630]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.numbers.One

#### Example 2

In [1631]:
d = log(3)/log(2)
a_i = [2]
b_i = [2]

In [1632]:
calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-0.709511291351455*n**1.0 + 1.70951129135146*n**1.58496250072116

In [1633]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.numbers.One

#### Example 3

In [1634]:
d = Rational(1/2)
a_i = [1]
b_i = [2]

In [1635]:
calculate_complexity(a_i, b_i, d)

2*sqrt(n) - 1

In [1636]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.numbers.Zero

#### Example 4

In [1637]:
d = 2
a_i = [2]
b_i = [2]

In [1638]:
calculate_complexity(a_i, b_i, d)

n**2

In [1639]:
power = calculate_p(a_i, b_i)
power

1

In [1640]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.numbers.One

#### Example 5

In [1641]:
d = 1
a_i = [1, 1]
b_i = [2, 4]

In [1642]:
calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-2.27055945395966*n**0.694241913630617 + 3.27055945395966*n**1.0

#### Example 6

In [1643]:
d = 1
a_i = [1, 2]
b_i = [2, 3]

In [1644]:
calculate_complexity(a_i, b_i, d)

multiple generators [2**p, 3**p, 6**p]
No algorithms are implemented to solve equation -1 + 2/3**p + 2**(-p)
Guess: 
2


-5.97768128075249*n**1.0 + 6.97768128075249*n**1.16728894583589

Example 7

In [1645]:
d = 1
a_i = [3]
b_i = [2]

In [1646]:
calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-1.70951129135146*n**1.0 + 2.70951129135145*n**1.58496250072116

### 3.2 Refactoring

In this section we will refactor the code a little bit and seperate the functionality for each major component ...

In [96]:
def objective_function(p):
    return abs(f(p) - 1.0)

In [97]:
def calculate_p(array_a_i, array_b_i):
    
    p = Symbol('p', finite=True, nonnegative=True, imaginary=False) # Set Attribute positive = true
    
    length = len(array_a_i)
    
    expression = 0
    for x in range(length):
        expression += a_i[x] / b_i[x]**p
    
    try:
        solution = solve(Eq(expression, 1), p)
   
    except NotImplementedError as e:
        print(e)
        f = lambdify(p, expression)
        
        print("Guess: ")
        guess = float(input())
        
        result = minimize(objective_function(p), [guess])
        
        solution = result.x
        for sol in result.x:
            round(sol)
    
    return solution[0]

In [98]:
def base_type(power, d):
    # test for base type of p or d
    if (
            (
                type(power) == type(log(2)/log(3))
                or
                type(power) == type(1+log(2))
            )
        
            or
        
            (
                type(d) == type(log(2)/log(3))
                or
                type(d) == type(1+log(2))
            )
       ):
        
        return true

In [99]:
def calculate_complexity(array_a_i, array_b_i, d):
    
    n = Symbol('n', positive=True, finite=True, integer=True)
    u = Symbol('u')
    
    power = calculate_p(array_a_i, array_b_i)
    
    # test for base type of p or d
    if (base_type(power, d)):
        
        ##`sympy.core.numbers.RationalConstant`
        print("Symbolic Integral Unsatisfactory - convert p to numerical value")
        power = N(power) # cast p to numerical value
        
        # convert d to numeric type
        d = N(d)

    integral = integrate(u**d / u**(power+1), (u, 1, n))

    result = expand(n**power + n**power * integral)
    
    return result

#### Example 1

In [100]:
d = log(3)/log(2)
a_i = [2]
b_i = [2]

calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-0.709511291351455*n**1.0 + 1.70951129135146*n**1.58496250072116

In [101]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.numbers.One

#### Example 2

In [102]:
d = 1
a_i = [2]
b_i = [2]

calculate_complexity(a_i, b_i, d)

n*log(n) + n

In [103]:
power = calculate_p(a_i, b_i)
power

1

In [104]:
type(power)

sympy.core.numbers.One

#### Example 3

In [105]:
d = 2
a_i = [1]
b_i = [2]

calculate_complexity(a_i, b_i, d)

n**2/2 + 1/2

In [106]:
power = calculate_p(a_i, b_i)
power

0

In [107]:
type(power)

sympy.core.numbers.Zero

#### Example 4

In [108]:
d = log(3)/log(2)
a_i = [2]
b_i = [3]

calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-0.0481820492929981*n**0.630929753571457 + 1.048182049293*n**1.58496250072116

In [109]:
power = calculate_p(a_i, b_i)
power

log(2)/log(3)

In [110]:
type(power)

sympy.core.mul.Mul

In [111]:
d

log(3)/log(2)

In [112]:
type(d)

sympy.core.mul.Mul

#### example 5

In [113]:
d = 1
a_i = [3]
b_i = [2]

calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-1.70951129135146*n**1.0 + 2.70951129135145*n**1.58496250072116

In [114]:
type(d)

int

In [115]:
type(power)

sympy.core.mul.Mul

In [116]:
power = calculate_p(a_i, b_i)
power

log(3)/log(2)

In [117]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.mul.Mul

### Some Exotic Examples ...

#### Example

In [118]:
d = 0.2
a_i = [3]
b_i = [2]

In [119]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.mul.Mul

In [120]:
calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-0.722041210126119*n**0.2 + 1.72204121012612*n**1.58496250072116

#### Example

In [121]:
d = log(log(2))
a_i = [3]
b_i = [2]

In [122]:
power = calculate_p(a_i, b_i)
power
type(power)

sympy.core.mul.Mul

In [123]:
calculate_complexity(a_i, b_i, d)

Symbolic Integral Unsatisfactory - convert p to numerical value


-0.5124327926879/n**0.366512920581664 + 1.5124327926879*n**1.58496250072116

## 4. Future Directions

### 4.1 Additional Features

This research will continue to expand in terms of ...

1) Unification
2) Visualization
3) Generalization

#### 1. Unification

A similar concept, but alternative approach has already been investigated and imlemented by the [big-O Python Library](https://github.com/pberkes/big_O) ...

The big-O python library has couple of benefits & drawbacks which this project aims to supplement/borrow ...

The big-O python library is based on an emirical approach to compute the runtime complexity for any given code-block/script by repeatedly running a given script and then infers the complexity based on the collected results. This approach is convenient if it is not known whether the implemented algorithm can be modelled by a Decrease/Divide & Conquer type recurrence relation. But it may be slow (potentially hang) due to repeatedly running the script for a multiple number of times, if the program is guaranteed to terminate in the first place. In a follow-up project for the above formalism, we would like to develop a software that would test a provided script for a recurrence relation (if it exists), in which case, the formalism in this document should be employed, otherwize, the software should default to the empirical implementation for estimating the runtime. In short, a future implementation of this project will aim at merging the solutions here with the Big-O Python Library

#### 2. Visualization

A future update based on this project should contain the functionality to graph the results of the analysis for a given recurrence relation for visual representation of the runtime complexity, giving developers an "intuitive tool" to work with. This implementation may potentially be based on [`manim`](https://www.manim.community/) or [`matplotlib`](https://matplotlib.org/) the decision for which is not yet finalized ...

#### 3 Generalization

There are many more aspects regarding the Akar-Bazzi Theorem which were not taken into account while developing this complexity calculator, due to these factors potentially convoluting the experiment, making it much more difficult to build for practical use. However, in a future release of this project, many additional features will be included to enable clients to "tweak" results as needed.

### 4.2 Possible Generalization

Besides the additional features that needs to be taken into consideration, a platora of potential research problems lies ahead to improve much of the features promised in the coming updates. For one, the close relationship of the Akar-Bazzi Integral ...

$$
    T(n) = n^p + n^p \int_{1}^{n} {\frac{u^d}{u^{p+1}}} du
$$

... to the [generalized powersum](https://en.wikipedia.org/wiki/Sums_of_powers) ...

$$
    \sum_{i=1}^{k} = i^1 + i^2 + i^3 \cdots + i^k
$$

... is a promising start for future research, considering that the integral term ...

$$
    \int_{1}^{n} {\frac{u^d}{u^{p+1}}} du = \int_{1}^{n} {u^{d - (p + 1)}} du = \int_{0}^{n} {u^{d - (p + 1)}} du - \int_{0}^{1} {u^{d - (p + 1)}} du
$$

... with ... 

$$\int_{0}^{n} {u^{d - (p + 1)}} du = \lim_{n \rightarrow \infty} \sum_{i=1}^{n} i^{d-(p+1)}$$

... and ...

$$\int_{0}^{1} {u^{d - (p + 1)}} du = \lim_{n \rightarrow \infty} \sum_{i=1}^{n} \left(\frac{i}{n}\right)^{d-(p+1)} \left( \frac{1}{n} \right) \approx \lim_{n \rightarrow \infty} \sum_{i=1}^{n} \left(\frac{i}{n}\right)^{d-(p+1)} $$

... where the expression ...

$$\int_{0}^{n} {u^{d - (p + 1)}} du = \lim_{n \rightarrow \infty} \sum_{i=1}^{n} i^{d-(p+1)}$$

... is a generalized power sum and a lot more can be said with regards to the convergence of these series ...

The point is to reconcile the Akar-Bazzi Theorem with the Master Theorem, which has a much more intuitive proof ...

At the very least, this leads us to an alternative approach for proving the Akar-Bazzi Theorem without reliance on [mathematical induction](https://en.wikipedia.org/wiki/Mathematical_induction), which is currently the most widely accepted form of proof.

## 5. Bibliography

1) [Wikipedia, Akra-Bazzi Theorem](https://en.wikipedia.org/wiki/Akra%E2%80%93Bazzi_method)

2) [Notes on a Better Master Theorem, Massachusetts Institute of Technology](https://courses.csail.mit.edu/6.046/fall02/handouts/akrabazzi.pdf)

3) [Libre Texts, Engineering - Divide-and-Conquer Recurrences](https://eng.libretexts.org/Bookshelves/Computer_Science/Programming_and_Computation_Fundamentals/Mathematics_for_Computer_Science_(Lehman_Leighton_and_Meyer)/05%3A_Recurrences/21%3A_Recurrences/21.04%3A_Divide-and-Conquer_Recurrences)

4) [big-O Python Library](https://github.com/pberkes/big_O)

5) [Wikipedia, Halting Problem](https://en.wikipedia.org/wiki/Halting_problem)